In [2]:
import Pkg
Pkg.add("Oscar")
Pkg.add("Polymake")
Pkg.add("Combinatorics")

  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`


In [1]:
using Oscar
using Polymake
using Combinatorics

 -----    -----    -----      -      -----   
|     |  |     |  |     |    | |    |     |  
|     |  |        |         |   |   |     |  
|     |   -----   |        |     |  |-----   
|     |        |  |        |-----|  |   |    
|     |  |     |  |     |  |     |  |    |   
 -----    -----    -----   -     -  -     -  

...combining (and extending) ANTIC, GAP, Polymake and Singular
Version 0.5.1 ... 
 ... which comes with absolutely no warranty whatsoever
Type: '?Oscar' for more information
(c) 2019-2021 by The Oscar Development Team


In [13]:
tv = fulton.NormalToricVariety(INPUT_RAYS=[0 1; 1 0; 0 -1; -2 1]; INPUT_CONES=[[0,1],[1,2],[2,3],[0,3]])

type: NormalToricVariety

INPUT_CONES
	{0 1}
	{1 2}
	{2 3}
	{0 3}

INPUT_RAYS
	0 1
	1 0
	0 -1
	-2 1

In [14]:
tv.SMOOTH

false

In [15]:
tv.SIMPLICIAL

true

In [16]:
tv.RAYS

pm::Matrix<pm::Rational>
0 1
1 0
0 -1
-1 1/2


In [17]:
tv.MAXIMAL_CONES

pm::IncidenceMatrix<pm::NonSymmetric>
{0 1}
{1 2}
{2 3}
{0 3}


In [18]:
function slicematrix(A::AbstractMatrix)
    return [A[i, :] for i in 1:size(A,1)]
end

slicematrix (generic function with 1 method)

In [19]:
function rowMinors(A::AbstractMatrix,S::Set)
    outList=[]
    slices=slicematrix(A)
    for i in 1:size(A,1)
        if i in S
            append!(outList,slices[i])
        end
    end
    return transpose(hcat(outList...))
end

rowMinors (generic function with 1 method)

In [21]:
# Get the rays of maximal cone i (tv.MAXIMAL_CONES[i,:] will get the incidence matrix booleans)
# findall gets the indices of the "true"s, then we take those specific rows
i = 3
coneRays = tv.RAYS[findall(bool -> bool, tv.MAXIMAL_CONES[i,:]),:]

pm::Matrix<pm::Rational>
0 -1
-1 1/2


In [57]:
# Seems to require cast to generic matrix in order to calculate rank
rank(Matrix(tv.RAYS))

2

In [59]:
rank(Matrix(coneRays))

2

In [1]:
function getConeRays(coneRayIndices, rayMatrix)
    return tv.RAYS[findall(bool -> bool, coneRayIndices),:]
end

getConeRays (generic function with 1 method)

In [2]:
function getConeRank(coneRayIndices, rayMatrix)
    coneRays = getConeRays(coneRayIndices, rayMatrix)
    return rank(Matrix(coneRays))
end

getConeRank (generic function with 1 method)

In [80]:
function getDimension(X)
    return size(X.RAYS, 2)
end

getDimension (generic function with 1 method)

In [81]:
getDimension(tv)

2

In [6]:
a = [1,2,3,4]
println(deleteat!(a, 3))
println(a)

[1, 2, 4]
[1, 2, 4]


In [7]:
function getConeFaces(coneRayIndices, rayMatrix)
    outCones = []
    # We want to find cones with dimension 1 less than the original
    # Calculate dimension of cone - 1
    c = getConeRank(coneRayIndices, rayMatrix) - 1
    for i in 1:size(coneRayIndices)
        # Iterate through the cone rays in order to iterate through
        # all subcones that are missing one ray from the original
        subcone = deleteat!(copy(coneRayIndices), i)
        
        # Check that it has dimension 1 less than the original
        if (getConeRank(subcone, rayMatrix) == c)
            push!(outCones, subcone)
        end
    end
    # for subcone in powerset(coneRayIndices, min=1, max=length(a)-1)
        # if (getConeRank(subcone, rayMatrix) == c)
            # push!(outCones, subcone)
    # end
    
    return outCones
    # This code may be incorrect, since I am not sure whether we
    # need to consider subcones missing more than 1 ray
end

getConeFaces (generic function with 1 method)

In [1]:
function makeSimplicial(X)
    Y = copy(X)
    while (true)
        # Maximal cones and ray matrix
        coneList = Y.MAXIMAL_CONES
        rayMatrix = Y.RAYS
        badCone = Nothing
        for i in 1:size(coneList,1)
            cone = coneList[i,:]
            if (getConeRank(cone, rayMatrix) != size(cone))
                badCone = cone
            end
        end
        
        if (badCone == Nothing)
            # All cones are linearly independent
            break
        else
            # Find the first ray that is contained in more than one orbit
            # and subdivide at that ray, using toricBlowup
            
            # Get faces
            edges = getConeFaces(badCone, rayMatrix)
            # Find the first ray that is contained in more than one orbit
            i = 1
            while count(r->(badCone[i] in r), edges) == 1
                i += 1
            end
            # Subdivide at the cone containing just that ray
            Y = toricBlowup([badCone[i]], Y)
        end
        # Repeat this process until there are no more bad cones
    end
    return Y
end
        

makeSimplicial (generic function with 1 method)